In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

In [2]:
features_df = pd.read_csv('main_SV.csv', low_memory=False)
features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 810 entries, SK_ID_CURR to per_NFLAG_INSURED_ON_APPROVAL_sum
dtypes: float64(427), int64(383)
memory usage: 1.9 GB


In [17]:
index = np.loadtxt('test_set_index.csv', dtype=int, delimiter=',')
test_data = features_df.iloc[index].sample(5000)
train_set = features_df.iloc[list(set(range(len(features_df))) - set(index))]

data_df_test = train_set.iloc[:, :]
data_df = data_df_test.sample(frac=1)
data_df.info()

train_data = data_df.iloc[0:247510, :].sample(50000)
train_data.reset_index(drop=True, inplace=True)

dev_data = data_df.iloc[247510:277510, :].sample(5000)
dev_data.reset_index(drop=True, inplace=True)

#test_data = data_df.iloc[277510:307510, :]
#test_data.reset_index(drop=True, inplace=True)

train_data.info()
dev_data.info()
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 277511 entries, 101695 to 245479
Columns: 810 entries, SK_ID_CURR to per_NFLAG_INSURED_ON_APPROVAL_sum
dtypes: float64(427), int64(383)
memory usage: 1.7 GB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Columns: 810 entries, SK_ID_CURR to per_NFLAG_INSURED_ON_APPROVAL_sum
dtypes: float64(427), int64(383)
memory usage: 309.0 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Columns: 810 entries, SK_ID_CURR to per_NFLAG_INSURED_ON_APPROVAL_sum
dtypes: float64(427), int64(383)
memory usage: 30.9 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 31439 to 181999
Columns: 810 entries, SK_ID_CURR to per_NFLAG_INSURED_ON_APPROVAL_sum
dtypes: float64(427), int64(383)
memory usage: 30.9 MB


In [9]:
print(train_data['TARGET'].sum()/ train_data.shape[0], dev_data['TARGET'].sum()/ dev_data.shape[0], test_data['TARGET'].sum()/ test_data.shape[0])

print(train_data.shape[0], dev_data.shape[0], test_data.shape[0])

0.08112399499010141 0.0774 0.0808
247510 30000 30000


In [5]:
def model(train_x,train_y,test_x,test_y,param,num_round):
    dtrain = xgb.DMatrix(train_x.values,label = train_y.values,feature_names=train_x.columns,missing=np.nan)
    dtest = xgb.DMatrix(test_x.values,label = test_y.values,feature_names=test_x.columns, missing=np.nan)
    watchlist  = [(dtrain,'dtrain'),(dtest,'dev')]
#     bst = xgb.train(param, dtrain, num_round, watchlist, early_stopping_rounds=20)
    bst = xgb.train(param, dtrain, num_round, watchlist)
    # 模型效果
    train_preds = bst.predict(dtrain, ntree_limit=num_round)
    train_labels = dtrain.get_label()
#     print('train : ', get_ks(train_preds, train_labels))
    print('train auc:', roc_auc_score(train_labels, train_preds))
    test_preds = bst.predict(dtest, ntree_limit=num_round)
    test_labels = dtest.get_label()
#     print('dev : ', get_ks(test_preds, test_labels))
    print('dev auc:', roc_auc_score(test_labels, test_preds))
    
    return train_preds,train_labels,test_preds,test_labels,bst

In [6]:
features_list = list(set(train_data.columns)-set(["TARGET"]))
len(features_list)

809

In [18]:
param = {
    'max_depth':6,
    'eval_metric':'auc', 
#     'objective':'binary:logitraw',
    'objective':'binary:logistic',
    'base_score': 0.2,
    'subsample':0.6,
    'tree_method':'approx',
    'lambda':700,
#     'booster':'dart',
#     'sample_type':'weighted',
#     'normalize_type':'tree',
#     'rate_drop':0.5,
#     'skip_drop':0.5,
    'seed':0,
#     'colsample_bytree':0.8,
#     'min_child_weight':700,
#     'gamma':10,
    'eta':0.4
}
train_preds, train_labels, dev_preds, dev_labels, bst = model(train_data.loc[:, features_list], train_data['TARGET'],
                                                              dev_data.loc[:, features_list], dev_data['TARGET'],
                                                              param, 500)


[18:37:54] Tree method is selected to be 'approx'
[18:37:55] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 108 extra nodes, 0 pruned nodes, max_depth=6
[0]	dtrain-auc:0.644653	dev-auc:0.62301
[18:37:56] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 112 extra nodes, 0 pruned nodes, max_depth=6
[1]	dtrain-auc:0.682125	dev-auc:0.655535
[18:37:56] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=6
[2]	dtrain-auc:0.70046	dev-auc:0.688148
[18:37:57] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 120 extra nodes, 0 pruned nodes, max_depth=6
[3]	dtrain-auc:0.712049	dev-auc:0.701947
[18:37:57] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 116 extra nodes, 0 pruned nodes, max_depth=6
[4]	dtrain-auc:0.726185	dev-

[18:38:14] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[43]	dtrain-auc:0.811852	dev-auc:0.781573
[18:38:14] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 116 extra nodes, 0 pruned nodes, max_depth=6
[44]	dtrain-auc:0.812656	dev-auc:0.781154
[18:38:15] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 102 extra nodes, 0 pruned nodes, max_depth=6
[45]	dtrain-auc:0.813599	dev-auc:0.78146
[18:38:15] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 120 extra nodes, 0 pruned nodes, max_depth=6
[46]	dtrain-auc:0.814616	dev-auc:0.780593
[18:38:16] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 118 extra nodes, 0 pruned nodes, max_depth=6
[47]	dtrain-auc:0.815708	dev-auc:0.781037
[18:38:16] C:\Users\Administrat

[18:38:33] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 104 extra nodes, 0 pruned nodes, max_depth=6
[86]	dtrain-auc:0.845173	dev-auc:0.783937
[18:38:33] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 112 extra nodes, 0 pruned nodes, max_depth=6
[87]	dtrain-auc:0.845908	dev-auc:0.783443
[18:38:33] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 98 extra nodes, 0 pruned nodes, max_depth=6
[88]	dtrain-auc:0.846441	dev-auc:0.782581
[18:38:34] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=6
[89]	dtrain-auc:0.847037	dev-auc:0.78299
[18:38:34] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 112 extra nodes, 0 pruned nodes, max_depth=6
[90]	dtrain-auc:0.84806	dev-auc:0.782946
[18:38:35] C:\Users\Administrator

[18:38:51] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 106 extra nodes, 0 pruned nodes, max_depth=6
[129]	dtrain-auc:0.87109	dev-auc:0.779671
[18:38:51] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 102 extra nodes, 0 pruned nodes, max_depth=6
[130]	dtrain-auc:0.87143	dev-auc:0.779392
[18:38:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 116 extra nodes, 0 pruned nodes, max_depth=6
[131]	dtrain-auc:0.872192	dev-auc:0.779748
[18:38:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 112 extra nodes, 0 pruned nodes, max_depth=6
[132]	dtrain-auc:0.872874	dev-auc:0.779556
[18:38:53] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=6
[133]	dtrain-auc:0.873287	dev-auc:0.779886
[18:38:53] C:\Users\Adminis

[18:39:09] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=6
[172]	dtrain-auc:0.891689	dev-auc:0.776649
[18:39:09] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=6
[173]	dtrain-auc:0.892104	dev-auc:0.777109
[18:39:09] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 108 extra nodes, 0 pruned nodes, max_depth=6
[174]	dtrain-auc:0.89245	dev-auc:0.77639
[18:39:10] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=6
[175]	dtrain-auc:0.892946	dev-auc:0.776403
[18:39:10] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 112 extra nodes, 0 pruned nodes, max_depth=6
[176]	dtrain-auc:0.893271	dev-auc:0.775928
[18:39:11] C:\Users\Adminis

...

[18:40:03] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 116 extra nodes, 0 pruned nodes, max_depth=6
[301]	dtrain-auc:0.935583	dev-auc:0.773735
[18:40:04] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 112 extra nodes, 0 pruned nodes, max_depth=6
[302]	dtrain-auc:0.935571	dev-auc:0.774054
[18:40:04] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 94 extra nodes, 0 pruned nodes, max_depth=6
[303]	dtrain-auc:0.935877	dev-auc:0.77412
[18:40:04] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 118 extra nodes, 0 pruned nodes, max_depth=6
[304]	dtrain-auc:0.936298	dev-auc:0.775006
[18:40:05] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 116 extra nodes, 0 pruned nodes, max_depth=6
[305]	dtrain-auc:0.93623	dev-auc:0.775199
[18:40:05] C:\Users\Administ

[18:40:21] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 106 extra nodes, 0 pruned nodes, max_depth=6
[344]	dtrain-auc:0.944996	dev-auc:0.772406
[18:40:22] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=6
[345]	dtrain-auc:0.945096	dev-auc:0.772179
[18:40:22] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=6
[346]	dtrain-auc:0.945443	dev-auc:0.772142
[18:40:23] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 104 extra nodes, 0 pruned nodes, max_depth=6
[347]	dtrain-auc:0.945693	dev-auc:0.772241
[18:40:23] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 102 extra nodes, 0 pruned nodes, max_depth=6
[348]	dtrain-auc:0.945878	dev-auc:0.77206
[18:40:24] C:\Users\Admini

[18:40:41] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=6
[387]	dtrain-auc:0.953577	dev-auc:0.769259
[18:40:41] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=6
[388]	dtrain-auc:0.953829	dev-auc:0.76946
[18:40:42] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 122 extra nodes, 0 pruned nodes, max_depth=6
[389]	dtrain-auc:0.953957	dev-auc:0.769269
[18:40:42] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 0 pruned nodes, max_depth=6
[390]	dtrain-auc:0.954241	dev-auc:0.769438
[18:40:42] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 120 extra nodes, 0 pruned nodes, max_depth=6
[391]	dtrain-auc:0.954452	dev-auc:0.769718
[18:40:43] C:\Users\Admini

[18:41:00] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 0 pruned nodes, max_depth=6
[430]	dtrain-auc:0.961458	dev-auc:0.766851
[18:41:01] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 104 extra nodes, 0 pruned nodes, max_depth=6
[431]	dtrain-auc:0.961727	dev-auc:0.766631
[18:41:01] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=6
[432]	dtrain-auc:0.961672	dev-auc:0.767161
[18:41:02] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 120 extra nodes, 0 pruned nodes, max_depth=6
[433]	dtrain-auc:0.961932	dev-auc:0.767734
[18:41:02] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 104 extra nodes, 0 pruned nodes, max_depth=6
[434]	dtrain-auc:0.962124	dev-auc:0.767702
[18:41:03] C:\Users\Admin

[18:41:20] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 116 extra nodes, 0 pruned nodes, max_depth=6
[473]	dtrain-auc:0.967193	dev-auc:0.766391
[18:41:21] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 122 extra nodes, 0 pruned nodes, max_depth=6
[474]	dtrain-auc:0.967385	dev-auc:0.766732
[18:41:21] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=6
[475]	dtrain-auc:0.967515	dev-auc:0.766308
[18:41:22] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 0 pruned nodes, max_depth=6
[476]	dtrain-auc:0.967611	dev-auc:0.766511
[18:41:22] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 120 extra nodes, 0 pruned nodes, max_depth=6
[477]	dtrain-auc:0.967854	dev-auc:0.766371
[18:41:22] C:\Users\Admin

In [19]:
dmatrix = xgb.DMatrix(test_data.loc[:, features_list].values,label = test_data['TARGET'].values,feature_names=features_list, missing=np.nan)
dpreds = bst.predict(dmatrix)
dlabels = dmatrix.get_label()
# test_data['pd1_score'] = dpreds
# print('ks : ', get_ks(dpreds, dlabels))
print(roc_auc_score(dlabels, dpreds))
for i in range(40):
    t = (i+1)/100
    y_bin = [1. if y_cont > t else 0. for y_cont in dpreds]
    print(t, f1_score(dlabels, y_bin))

0.7567724492806753
0.01 0.15872349359647284
0.02 0.18257046806309798
0.03 0.20738900962434026
0.04 0.22428571428571428
0.05 0.2400655469070053
0.06 0.2532467532467533
0.07 0.2690302398331596
0.08 0.27958236658932717
0.09 0.29240506329113924
0.1 0.28687415426251694
0.11 0.2883787661406026
0.12 0.2874711760184474
0.13 0.28641171684296174
0.14 0.2926829268292683
0.15 0.2986175115207373
0.16 0.2950819672131148
0.17 0.29116465863453816
0.18 0.2925809822361546
0.19 0.28854625550660795
0.2 0.2919042189281642
0.21 0.3010752688172043
0.22 0.2917181705809641
0.23 0.28791773778920304
0.24 0.2826666666666666
0.25 0.281767955801105
0.26 0.2693409742120344
0.27 0.26647144948755486
0.28 0.2620481927710843
0.29 0.2561728395061728
0.3 0.2543720190779014
0.31 0.24183006535947715
0.32 0.24201680672268908
0.33 0.22797927461139897
0.34 0.2206405693950178
0.35 0.21493624772313294
0.36 0.2136279926335175
0.37 0.21052631578947367
0.38 0.20229007633587784
0.39 0.18992248062015504
0.4 0.1811023622047244


In [11]:
bst.save_model('xgb_.model')

In [12]:
bst.load_model("test1.model")

In [10]:
feature_score = bst.get_fscore()
feature_score = sorted(feature_score.items(), key=lambda x:x[1],reverse = True)  
feature_score

[('EXT_SOURCE_1', 3256),
 ('EXT_SOURCE_3', 3112),
 ('EXT_SOURCE_2', 3018),
 ('DAYS_BIRTH', 2505),
 ('AMT_ANNUITY', 1776),
 ('DAYS_ID_PUBLISH', 1531),
 ('per_CNT_PAYMENT_mean', 1385),
 ('AMT_CREDIT', 1353),
 ('DAYS_EMPLOYED', 1315),
 ('flag_pay_on_time_mean', 1042),
 ('AMT_PAYMENT_sum', 1032),
 ('OWN_CAR_AGE', 1005),
 ('DAYS_REGISTRATION', 947),
 ('AMT_GOODS_PRICE', 946),
 ('per_DAYS_LAST_DUE_1ST_VERSION_sum', 921),
 ('DAYS_LAST_PHONE_CHANGE', 914),
 ('AMT_payment_instalment_p', 909),
 ('per_DAYS_LAST_DUE_1ST_VERSION_max', 865),
 ('days_pay_days_mean', 821),
 ('per_HOUR_APPR_PROCESS_START_mean', 754),
 ('per_NAME_YIELD_GROUP_high_count_norm', 712),
 ('per_AMT_ANNUITY_mean', 703),
 ('per_AMT_ANNUITY_min', 697),
 ('SK_ID_CURR', 639),
 ('cc_CNT_DRAWINGS_ATM_CURRENT_mean', 633),
 ('days_pay_days_early', 621),
 ('SK_DPD_DEF_mean', 616),
 ('complete_percentage', 604),
 ('per_DAYS_DECISION_mean', 597),
 ('flag_pay_early_mean', 591),
 ('REGION_POPULATION_RELATIVE', 588),
 ('per_DAYS_DECISION_ma